In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


from data import get_data
from time import time
import joblib


In [ ]:
def get_data(urls, sheets, types):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        s = pd.read_excel(urls, sheet_name=s)
        s = s[[types, 'description']]
        s['description'] = s['description'].str.lower()
        if i == 0:
            data = s.copy()
        else:
            data = pd.concat([data, s], ignore_index=True)
        i += 1
    data.columns = ['target', 'data']
    data['target'] = data['target'].apply(str)
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

    return data

## Get Data

In [4]:
import pandas as pd

def loads_dataset(urls, sheets):
    data = ""
    t0 = time()
    a = []
    i = 0
    for s in sheets:
        s = pd.read_excel(urls, sheet_name=s)
        s = s[[ 'description','section','chapter','heading','sub-heading','tariff']]
        s['section'] = s['section'].map('{:02}'.format)
        s['chapter'] = s['chapter'].map('{:02}'.format)
    #s['heading'] = s['heading'].apply('{:04}'.format)
    #s['sub-heading'] = s['sub-heading'].map('{:06}'.format)
    #s['tariff'] = s['tariff'].map('{:08}'.format)
    s['description'] = s['description'].str.lower()
    
    if i == 0:
            data = s.copy()
    else:
            data = pd.concat([data, s], ignore_index=True)
    i += 1
   # data.columns = ['target', 'data']
  #  data['target'] = data['target'].apply(str)
    load_time = time() - t0
    print("Load dataset time:  %0.3fs" % load_time)

    return data   



In [5]:
urls = 'data/hs_code.xlsx'
types = 'section'

sheets = ['8_digit','6_digit','4_digit','2_digit', 'test_01', 'Declaration_2019_10']

df = loads_dataset(urls, sheets)
df.sample(10)

Load dataset time:  125.177s


,description,section,chapter,heading,sub-heading,tariff
2907,aloe vera drink lychee,04,22,2202,220210,22021090
1482,aluminium door frame g180182aada,15,76,7610,761010,76101010
9898,assorted ornaments / sterling silver 925rings,14,71,7113,711311,71131190
4125,aluminium fabrication bar ejwagz0825,15,76,7610,761090,76109099
18800,aircondition haier hsu09ctb03tfinoutท่อ/9000btu,16,84,8415,841510,84151090
18949,ab39 7a149 aa housing,17,87,8708,870830,87083029
3943,agi panter small 2x20s,04,24,2402,240210,24021000
3549,automotive disc brake pads 220027780 set tdb12...,17,87,8708,870830,87083090
9547,availa zn 120 animal feed additives,04,23,2309,230990,23099020
1993,aluminium door frame wf157182asaa,15,76,7610,761010,76101010


In [2]:
urls = 'data/hs_code.xlsx'
types = 'section'

sheets = ['8_digit','6_digit','4_digit','2_digit', 'test_01', 'Declaration_2019_10']

df = get_data(urls,sheets,types)
print(len(df))
df.sample(10)

Load dataset time:  126.454s
49137


,target,data
9418,17,vehicles; with both sparkignition internal com...
47631,17,accessory parts part number 08413b1010 toyota ...
3825,9,wood; charcoal of wood other than bamboo (incl...
45600,16,assy shelf glassref
8386,16,"transformers; n.e.c. in item no. 8504.2, havin..."
49,1,"meat; of bovine animals, cuts with bone in (ex..."
8382,16,"transformers; n.e.c. in item no. 8504.2, havin..."
163,1,"fish; fresh or chilled, southern bluefin tunas..."
6245,15,"steel, stainless; wire - other"
33248,15,arm/baserwindow 1d0058560


In [3]:
#save the DataFrame
print("Save DataFrame")

joblib.dump(df, 'data/df_master_train.pkl', compress=1)
print('success')

Save DataFrame
success


##  Create Model

In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)    
    return string.strip().lower()

In [4]:
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

def create_model(df,names):
    X = []
    for i in range(df.shape[0]):
        X.append(clean_str(df.iloc[i][1]))
    y = np.array(df["target"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)



    print("pipeline of feature engineering and model")
    model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

    print("paramater selection")


    parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

    print("Training: ")
   
    t0 = time()
    gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
    gs_clf_svm = gs_clf_svm.fit(X, y)
    print(gs_clf_svm.best_score_)
    print(gs_clf_svm.best_params_)

    model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

#fit model with training data
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    print(accuracy_score(y_test, pred))

#save the model
    print("Save Model")

    joblib.dump(model, 'data/model_'+names +'.pkl', compress=1)


    print (classification_report(y_test, pred))

In [6]:
#['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
 #      '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']

lst =[ '01', '02', '03']

for x in lst:
    names = x
    df = joblib.load('data/df_master_section.pkl')
    
    df[df['target'] == f'{x}']
    
    #save the DataFrame
    
    joblib.dump(df, 'data/train/df_master_'+x+'.pkl', compress=1)
    print("Save DataFrame")   
    create_model(df,names)
    print('success')

Save DataFrame
pipeline of feature engineering and model
paramater selection
Training: 
0.8787852808408421
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}
0.9354904354904355
Save Model
              precision    recall  f1-score   support

           1       0.99      0.99      0.99       180
          10       0.94      0.95      0.95        85
          11       0.98      0.98      0.98       403
          12       0.98      0.91      0.94        45
          13       0.95      0.92      0.94       104
          14       0.91      0.89      0.90        55
          15       0.97      0.96      0.96      1039
          16       0.90      0.93      0.91       855
          17       0.86      0.91      0.88       518
          18       0.94      0.89      0.92       147
          19       1.00      0.88      0.93         8
           2       0.99      1.00      0.99       155
          20       0.89      0.90      0.89       109
          21       1.00      1.00      1.00   

In [7]:
import joblib
print("Load Model")
model = joblib.load('data/model/model_01.pkl')
products = input()  # Video game consoles : 95

Load Model
Vanilla 


In [8]:
model.predict([products])[0]  # Vanilla : 02

'2'

In [9]:
import joblib
import os

%time df = joblib.load('data/train/df_master_02.pkl')
df

Wall time: 95.9 ms


,target,data
0,1,"horses; live, purebred breeding animals - pure..."
1,1,"horses; live, other than purebred breeding ani..."
2,1,asses; live - other
3,1,mules and hinnies; live- other
4,1,"cattle; live, purebred breeding animals - pure..."
...,...,...
49132,15,adjust gear set jfkz658b
49133,15,aluminium window frame kf057072avat
49134,7,acrylic plate 4mm. size24x24 cm.
49135,16,ac power cord p/n 141102240p6


In [12]:
x= "1"
df[df['target'] == f'{x}']
df

,target,data
0,1,"horses; live, purebred breeding animals - pure..."
1,1,"horses; live, other than purebred breeding ani..."
2,1,asses; live - other
3,1,mules and hinnies; live- other
4,1,"cattle; live, purebred breeding animals - pure..."
...,...,...
49132,15,adjust gear set jfkz658b
49133,15,aluminium window frame kf057072avat
49134,7,acrylic plate 4mm. size24x24 cm.
49135,16,ac power cord p/n 141102240p6
